In [ ]:
!pip install sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
import sacrebleu
import pandas as pd
import numpy as np

In [ ]:
# The project folder can be saved in google drive and accessed through Google colab
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path to the project folder that contains a folder code, data and models
path_project = "/content/drive/MyDrive/anlp_project/NLLB-200/ANLP_SUBMISSION"

import os
os.chdir(path_project)

In [ ]:
def evaluate(df, columns_to_evaluate, ref_col="de"):
    for column in columns_to_evaluate:
        bleu_calc = sacrebleu.BLEU()
        chrf_calc = sacrebleu.CHRF(word_order=2)
        ter_calc = sacrebleu.TER()
        bleu = bleu_calc.corpus_score(df[column].tolist(), [df[ref_col].tolist()], n_bootstrap=1000)
        chrf = chrf_calc.corpus_score(df[column].tolist(), [df[ref_col].tolist()], n_bootstrap=1000)
        ter = ter_calc.corpus_score(df[column].tolist(), [df[ref_col].tolist()], n_bootstrap=1000)
        print(column)
        print(bleu)
        print(chrf)
        print(ter)
        print("---------------------")

# Test Set

In [ ]:
df_test = pd.read_excel(f"data/Results/df_test_Results.xlsx")
df_test.head()

In [ ]:
columns = ['init_none', 'init_vs', 'init_gr_large', 'init_de', 'init_gr_small']
evaluate(df_test, columns)

In [ ]:
# Save the column to a text file (e.g., CSV) to use through sacrebleu command line tools
for column in columns:
    df_test[column].to_csv(f'data/Results/text_files/{column}.txt', index=False, header=False)
df_test["de"].to_csv(f'data/Results/ref_de.txt', index=False, header=False)

In [ ]:
%%bash
path="data/Results/text_files/"
sacrebleu "${path}ref_de.txt" -i "${path}/init_none.txt" "${path}/init_vs.txt" "${path}/init_gr_large.txt" "${path}/init_de.txt" "${path}/init_gr_small.txt" -m bleu chrf ter --paired-bs --format text

In [ ]:
%%bash
path="data/Results/text_files/"
sacrebleu "${path}ref_de.txt" -i "${path}/init_vs.txt" "${path}/init_gr_small.txt" -m bleu chrf ter --paired-bs --format text

In [ ]:
%%bash
path="data/Results/text_files/"
sacrebleu "${path}ref_de.txt" -i "${path}/init_vs.txt" "${path}/init_gr_large.txt" -m bleu chrf ter --paired-bs --format text

In [ ]:
%%bash
path="data/Results/text_files/"
sacrebleu "${path}ref_de.txt" -i "${path}/init_de.txt" "${path}/init_vs.txt" "${path}/init_gr_large.txt" "${path}/init_gr_small.txt" -m bleu chrf ter --paired-bs --format text

# Synthetic Test Set

In [ ]:
df_syn = pd.read_excel("data/Results/Manual_Syntax_Evaluation_Synt_TestSet.xlsx")

In [ ]:
df_syn.columns

In [ ]:
evaluate(df_syn, ['init_none', 'init_vs', 'init_gr_large', 'init_de', 'init_gr_small'])

In [ ]:
df_syn.columns

In [ ]:
cols = ["num_init_none", "num_init_vs", "num_init_gr_large", "num_init_de", "num_init_gr_small"]

# store only the columns with the numeric evaluation
df_syn_sub = df_syn[cols]

In [ ]:
def eval_measure(array):
    """
    Function to get the accuracy for each column
    """
    return np.sum(array, axis=0)/ array.shape[0]

# print the accuracy for all columns
eval_measure(df_syn_sub)

In [ ]:
def eval_with_paired_bootstrap(df, cols, num_samples=1000, sample_ratio=0.6):
    """
    Function to perform paired bootstrap resampling.
    """

    df = df[cols]

    array = np.array(df)
    sys1_scores = []
    sys2_scores = []
    wins = [0, 0, 0]

    print(eval_measure(array))

    for _ in range(num_samples):

        # shuffles along the first axis
        np.random.shuffle(array)

        # take the sample ratio
        ratio_samples = int(array.shape[0]*sample_ratio)

        # reduce the array
        array_reduced = array[:ratio_samples]

        # Calculate accuracy on the reduced sample and save stats
        score = eval_measure(array_reduced)

        if score[0] > score[1]:
            wins[0] += 1
        elif score[0] < score[1]:
            wins[1] += 1
        else:
            wins[2] += 1

        sys1_scores.append(score[0])
        sys2_scores.append(score[1])

    # Print win stats
    wins = [x/float(num_samples) for x in wins]
    print(f'Win ratio: sys1={wins[0]}, sys2={wins[1]}, tie={wins[2]}')
    if wins[0] > wins[1]:
        p_value = 1-wins[0]
        print('(sys1 is superior with p value p=%.3f)' % (p_value))
    elif wins[1] > wins[0]:
        p_value = 1-wins[1]
        print('(sys2 is superior with p value p=%.3f)' % (p_value))

    # Print system stats
    sys1_scores.sort()
    sys2_scores.sort()
    print('sys1 mean=%.3f, median=%.3f, 95%% confidence interval=[%.3f, %.3f]' %
        (np.mean(sys1_scores), np.median(sys1_scores), sys1_scores[int(num_samples * 0.025)], sys1_scores[int(num_samples * 0.975)]))
    print('sys2 mean=%.3f, median=%.3f, 95%% confidence interval=[%.3f, %.3f]' %
        (np.mean(sys2_scores), np.median(sys2_scores), sys2_scores[int(num_samples * 0.025)], sys2_scores[int(num_samples * 0.975)]))

    return p_value

In [ ]:
# define the hypothesis which for the comparison

sq0 = [["num_init_none", "num_init_vs"],
       ["num_init_none", "num_init_gr_large"],
       ["num_init_none", "num_init_de"],
       ["num_init_none", "num_init_gr_small"]
      ]

sq1 = [["num_init_gr_small", "num_init_vs"]]

sq2 = [["num_init_gr_large", "num_init_vs"]]

sq3 = [["num_init_de", "num_init_vs"],
       ["num_init_de", "num_init_gr_large"],
       ["num_init_de", "num_init_gr_small"]
      ]

sqs = {"sq0": sq0, "sq1": sq1, "sq2": sq2, "sq3": sq3}

In [ ]:
pairs = [" ".join(pair) for sq in sqs.values() for pair in sq]

df_p_values = pd.DataFrame({"pairs": pairs})

pairs = []
p_values = []
s_ratio = 0.6

for hypothesis, sq in sqs.items():

    for pair in sq:
        print(f"{hypothesis}\nPair: {pair}\n------------------")
        p = eval_with_paired_bootstrap(df_syn_sub, cols=pair, sample_ratio=s_ratio)
        pairs.append(" ".join(pair))
        p_values.append(p)

df_p_values[str(s_ratio)] = p_values

In [ ]:
df_p_values